In [2]:
import xml.etree.ElementTree as ET
from time import time
import numpy as np
from scipy import io
import os
import pandas as pd

In [3]:
def readxml(xml_path):
    print(os.path.basename(xml_path))
    tree = ET.parse(xml_path)
    root = tree.getroot()

    classlut = []
    for Annotation in root.iter('Annotation'):
        for Attrib in Annotation.iter('Attribute'):
            classlut.append(Attrib.attrib.get('Name'))
    # print(classlut)
    # print(len(classlut),xml_path)
    if not (classlut==['collagen', 'non-collagen', 'tissue', 'epidermis'] or classlut==['collagen', 'tissue', 'epidermis'] or classlut==['non-collagen', 'collagen', 'tissue', 'epidermis']):
        print(classlut)

    return

src =  r'\\10.162.80.6\Kyu_Sync\Research\Active\Aging_organized\svs\svs_back'
[readxml(os.path.join(src,xml_path)) for xml_path in os.listdir(src) if xml_path.endswith('xml') ]

wf16_b2_01.xml
wf21_b4_01.xml
wf24_b5_01.xml
wf25_b4_01.xml
wf26_b5_01.xml
wf27_b5_01.xml
wf28_b4_01.xml
wf29_b3_01.xml
wf29_b4_01.xml
wf30_b3_01.xml
wf33_b3_01.xml
wf35_b3_01.xml
wf36_b3_01.xml
['Description', 'Description']
wf37_b3_01.xml
wf38_b3_01.xml
wf38_b3_02.xml
wf38_b3_03.xml
wf40_b3_01.xml
wf40_b3_02.xml
wf41_b4_01.xml
wf42_b4_01.xml
wf45_b4_01.xml
wf45_b5_01.xml
wf46_b5_01.xml
wf51_b4_01.xml
wf53_b5_01.xml
wf54_b4_01.xml
wf56_b5_01.xml
wf59_b5_01.xml
wf61_b4_01.xml
wf63_b4_01.xml
wf64_b3_01.xml
wf64_b4_01.xml
wf65_b4_01.xml
wf69_b3_01.xml
wf73_b3_01.xml
wf74_b3_01.xml
wf82_b3_01.xml
wf85_b3_01.xml
wf86_b3_02.xml
['non-collagen', 'collagen', 'tissue', 'epidermis', 'Description']
wf87_b3_01.xml
wf87_b3_03.xml
wf92_b3_01.xml
wm14_b2_01.xml
wm14_b2_02.xml
wm17_b2_01.xml
wm20_b2_02.xml
wm21_b2_01.xml
wm25_b4_01.xml
wm27_b4_01.xml
wm27_b4_02.xml
wm28_b4_01.xml
wm30_b4_01.xml
wm31_b4_01.xml
wm36_b4_01.xml
wm36_b5_01.xml
['corneum', 'basale', 'hairroot', 'hairfollicle', 'smoothmuscl

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]